In [1]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict


In [2]:
dataset = pd.read_csv(r'C:/Users/Pedro Henrique/PGC/input/NoThemeTweets.csv',encoding='utf-8')
dataset.count()

id            785814
tweet_text    785814
tweet_date    785814
sentiment     785814
query_used    785814
dtype: int64

In [3]:
# Os classificados como positivo
dataset[dataset.sentiment == 'Positivo'].count()

id            263107
tweet_text    263107
tweet_date    263107
sentiment     263107
query_used    263107
dtype: int64

In [5]:
#Os classificados como negativo
dataset[dataset.sentiment == 'Negativo'].count()

id            522707
tweet_text    522707
tweet_date    522707
sentiment     522707
query_used    522707
dtype: int64

In [6]:
dataset.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:)
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:)
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:)
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:)
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:)


In [7]:
tweets = dataset["tweet_text"].values
tweets

array(['@Tixaa23 14 para eu ir :)',
       '@drexalvarez O meu like eu já dei na época :)',
       'Eu só queria conseguir comer alguma coisa pra poder dormir :)',
       ...,
       'Opa tava na merda mm e fiquei logo mais feliz só por causa daquilo :)',
       '@andrebraga2806 Foi como a tua lealdade :)',
       'Feliz dia das crianças!! De hoje e de ontem... :) https://t.co/m7dAEUEM3q'],
      dtype=object)

In [9]:
classes = dataset["sentiment"].values
classes

array(['Positivo', 'Positivo', 'Positivo', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype=object)

Pré-Processamento

In [18]:
import spacy
import re

# Carregar o modelo de português
nlp = spacy.load('pt_core_news_sm')

# Função para remover emojis e emoticons
def remove_emoji_emoticon(text):
    # Remover emojis usando regex
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # Emoticons
                           u"\U0001F300-\U0001F5FF"  # Símbolos e pictogramas diversos
                           u"\U0001F680-\U0001F6FF"  # Transportes e símbolos de mapa
                           u"\U0001F1E0-\U0001F1FF"  # Bandeiras
                           "]+", flags=re.UNICODE)

    # Remover emoticons comuns (como :), :D, :(, =), =D, =(, etc)
    emoticon_pattern = re.compile(r'(:\)|:\(|:D|:P|D:|:-\)|:-\(|:-D|:-P|D:-|:\'\(|;\)|:-\]|:-\[|:o|:-o|:O|:-O|'
                                  r'=\)|=\(|=D|:-\)|:-\(|:-D|:-P|=\-\)|=\-\(|=\-\[|=\-\]|=\-\])')

    # Remover ambos
    text = emoji_pattern.sub(r'', text)
    text = emoticon_pattern.sub(r'', text)
    
    return text

# Função para pré-processar tweets com spaCy
def preprocess_tweet_spacy(tweet):
    doc = nlp(tweet)
    tokens = []
    for token in doc:
        # Remover menções e links
        if token.text.startswith('@') or token.like_url:
            continue
        tokens.append(token.text)
    cleaned_tweet = ' '.join(tokens)
    # Remover emojis, emoticons e converter para minúsculas
    cleaned_tweet = remove_emoji_emoticon(cleaned_tweet).lower()
    return cleaned_tweet.strip()

# Aplicar o pré-processamento a todos os tweets
tweets_preprocessed = [preprocess_tweet_spacy(tweet) for tweet in dataset['tweet_text']]

# Adicionar a coluna de tweets pré-processados ao DataFrame
dataset['tweet_text_preprocessed'] = tweets_preprocessed

# Visualizando as primeiras linhas do DataFrame com a nova coluna
dataset[['tweet_text', 'tweet_text_preprocessed']].head()

,tweet_text,tweet_text_preprocessed
0,@Tixaa23 14 para eu ir :),14 para eu ir
1,@drexalvarez O meu like eu já dei na época :),o meu like eu já dei na época
2,Eu só queria conseguir comer alguma coisa pra ...,eu só queria conseguir comer alguma coisa pra ...
3,:D que lindo dia !,que lindo dia !
4,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...","pq da pr jeito!!é uma "" oferta "" , ha q aprove..."
